In [12]:
# Import Data 
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data import EquityPricing, Fundamentals
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import SimpleMovingAverage, RSI
import quantopian.algorithm as algo
import numpy as np



In [62]:
def initialize(context):
    my_pipe = make_pipeline()
    algo.attach_pipeline(my_pipe, 'my_pipeline')

In [3]:
class StandardDeviation(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        out[:] = np.nanstd(values, axis=0)

In [63]:
def make_pipeline():
    
    base_universe = QTradableStocksUS()
    
    # This is a FACTOR, which is like a function. We do not get the result until we add this to our pipe 
    sma_20 = SimpleMovingAverage(
                inputs = [EquityPricing.close],
                window_length = 20
    )
    
    last_price =  EquityPricing.close.latest
    
    price_sma_ratio = last_price/sma_20
    
    price_std = StandardDeviation(
                inputs = [EquityPricing.close],
                window_length = 20
    )
    
    BB_High = sma_20 + 2*price_std
    BB_Low = sma_20 - 2*price_std
    BB_Percent = (last_price - BB_Low) / (BB_High - BB_Low)
    
    rsi = RSI(
        inputs = [EquityPricing.close],
        window_length = 14
    )
    
    # Strategy here, built on Factors
    longs = (BB_Percent < -.05) & (price_sma_ratio < 0.95) & (rsi < 30)
    shorts = (BB_Percent > 1.05) & (price_sma_ratio > 1.05) & (rsi > 70)
    
    # Filter to only securities with activity 
    securities_to_trade = (shorts | longs)
    
    
    return Pipeline(
        columns = {
            "20_Day_SMA": sma_20,
            "Latest_Price": last_price,
            "Price_SMA_Ratio": price_sma_ratio,
            "STD": price_std,
            "BB_High":BB_High,
            "BB_Low":BB_Low,
            "BB_Percent":BB_Percent,
            "RSI":rsi,
            "Longs": longs,
            "Shorts": shorts,
        },
    #domain=US_EQUITIES
        screen = securities_to_trade
    )

In [64]:
#my_pipe = make_pipeline()

In [65]:
result = run_pipeline(my_pipe, '2010-01-01','2011-12-31')

In [66]:
# This will show an index for all our securities over the time period

# This is a Multi-Index dataframe, with date and Symbol in Index
result.head(20)

20_Day_SMA     BB_High  \
2010-01-04 00:00:00+00:00 Equity(192 [ATAX])        5.484025    5.815442   
                          Equity(332 [ALX])       278.397000  300.630459   
                          Equity(1323 [CAW])        4.661050    5.486406   
                          Equity(1653 [MOC])        2.143684    2.473917   
                          Equity(2078 [DAIO])       3.957000    4.337899   
                          Equity(2796 [FFHS])       7.225444    7.900483   
                          Equity(2886 [FLXS])       8.869607   10.002571   
                          Equity(3070 [FUR])        9.336078   10.472301   
                          Equity(4319 [KTCC])       3.441400    3.805818   
                          Equity(4833 [MGIC])       1.957400    2.252781   
                          Equity(5062 [FOH])        1.167450    1.368325   
                          Equity(5676 [OPTC])       2.194400    2.565235   
                          Equity(5816 [PMD])        5.950050    7.016219   
                          Equity(6320 [QUIX])       2.379500    4.376162   
                          Equity(6693 [SCE_PRD])   18.749545   21.081172   
                          Equity(6803 [SFD])       16.061450   16.802673   
                          Equity(6804 [SFE])        9.346500   10.079883   
                          Equity(7633 [TTI])        9.924000   10.918372   
                          Equity(8910 [ANIK])       8.569000    9.220150   
                          Equity(10245 [PAL])       3.004000    3.370791   

                                                      BB_Low  BB_Percent  \
2010-01-04 00:00:00+00:00 Equity(192 [ATAX])        5.152609    1.248264   
                          Equity(332 [ALX])       256.163541    1.098265   
                          Equity(1323 [CAW])        3.835694    1.069421   
                          Equity(1653 [MOC])        1.813451    1.266604   
                          Equity(2078 [DAIO])       3.576101    1.160280   
                          Equity(2796 [FFHS])       6.550406    1.066305   
                          Equity(2886 [FLXS])       7.736642    1.100369   
                          Equity(3070 [FUR])        8.199856    1.144205   
                          Equity(4319 [KTCC])       3.076982    1.060620   
                          Equity(4833 [MGIC])       1.662019    1.063001   
                          Equity(5062 [FOH])        0.966575    1.477102   
                          Equity(5676 [OPTC])       1.823565    1.330018   
                          Equity(5816 [PMD])        4.883881    1.147154   
                          Equity(6320 [QUIX])       0.382838    1.499293   
                          Equity(6693 [SCE_PRD])   16.417919    1.278524   
                          Equity(6803 [SFD])       15.320227   -0.081100   
                          Equity(6804 [SFE])        8.613117    1.156887   
                          Equity(7633 [TTI])        8.929628    1.086300   
                          Equity(8910 [ANIK])       7.917850   -0.221032   
                          Equity(10245 [PAL])       2.637209    1.162502   

                                                  Latest_Price  Longs  \
2010-01-04 00:00:00+00:00 Equity(192 [ATAX])             5.980  False   
                          Equity(332 [ALX])            305.000  False   
                          Equity(1323 [CAW])             5.601  False   
                          Equity(1653 [MOC])             2.650  False   
                          Equity(2078 [DAIO])            4.460  False   
                          Equity(2796 [FFHS])            7.990  False   
                          Equity(2886 [FLXS])           10.230  False   
                          Equity(3070 [FUR])            10.800  False   
                          Equity(4319 [KTCC])            3.850  False   
                          Equity(4833 [MGIC])            2.290  False   
                          Equity(5062 [FOH])         